In [1]:
1+1

2

In [2]:
# cell 1: 라이브러리 설치 및 import
# !pip install langchain openai python-dotenv mysql-connector-python pandas sqlalchemy

import os
from dotenv import load_dotenv
import pandas as pd
import mysql.connector
from sqlalchemy import create_engine, text
from typing import List, Dict, Any, Optional
import json
import logging
from datetime import datetime

# LangChain imports
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain.chains import LLMChain
from langchain.schema import BaseOutputParser
from langchain.output_parsers import PydanticOutputParser, OutputFixingParser
from langchain.callbacks import get_openai_callback

# Pydantic for structured output
from pydantic import BaseModel, Field, validator
from enum import Enum

# 환경 변수 로드
load_dotenv()

# 로깅 설정
logging.basicConfig(
    level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s"
)
logger = logging.getLogger(__name__)

# OpenAI API 키 설정
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY", "your-openai-api-key-here")

print("✅ 라이브러리 import 및 환경 설정 완료")
print(f"📊 OpenAI API 키 설정: {'✅' if os.getenv('OPENAI_API_KEY') else '❌'}")

✅ 라이브러리 import 및 환경 설정 완료
📊 OpenAI API 키 설정: ✅


In [3]:
# cell 2: 데이터 모델 정의 (Pydantic)
class PolicyCategory(str, Enum):
    """정책 카테고리 열거형"""

    SCHOLARSHIP = "장학금"
    EMPLOYMENT = "취업지원"
    STARTUP = "창업지원"
    HOUSING = "주거지원"
    LOAN = "대출지원"
    EDUCATION = "교육지원"
    WELFARE = "복지지원"
    MEDICAL = "의료지원"
    CULTURE = "문화지원"
    GENERAL = "일반지원"


class AgeRange(BaseModel):
    """연령 범위 모델"""

    min_age: Optional[int] = Field(None, description="최소 연령")
    max_age: Optional[int] = Field(None, description="최대 연령")


class QueryIntent(BaseModel):
    """쿼리 의도 분석 결과"""

    category: PolicyCategory = Field(..., description="정책 카테고리")
    keywords: List[str] = Field(default_factory=list, description="검색 키워드")
    age_range: Optional[AgeRange] = Field(None, description="연령 범위")
    target_group: Optional[str] = Field(
        None, description="대상 그룹 (대학생, 청년, 고등학생 등)"
    )
    region: Optional[str] = Field(None, description="지역 제한")
    additional_conditions: Dict[str, Any] = Field(
        default_factory=dict, description="추가 조건"
    )

    @validator("keywords")
    def validate_keywords(cls, v):
        """키워드 유효성 검사"""
        return [keyword.strip() for keyword in v if keyword.strip()]


class MultiQueryAnalysis(BaseModel):
    """다중 쿼리 분석 결과"""

    is_multiple: bool = Field(..., description="다중 쿼리 여부")
    sub_queries: List[str] = Field(default_factory=list, description="분리된 서브 쿼리")
    intents: List[QueryIntent] = Field(
        default_factory=list, description="각 서브 쿼리의 의도"
    )


class SQLQuery(BaseModel):
    """생성된 SQL 쿼리"""

    query: str = Field(..., description="생성된 SQL 쿼리")
    explanation: str = Field(..., description="쿼리 설명")
    estimated_results: int = Field(default=0, description="예상 결과 수")


print("✅ 데이터 모델 정의 완료")

✅ 데이터 모델 정의 완료


C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_13712\2512926979.py:38: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_validator` validators, see the migration guide for more details. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  @validator("keywords")


In [4]:
# cell 3: 데이터베이스 스키마 정보 (LangChain에서 사용할 메타데이터)
DATABASE_SCHEMA = """
-- 온통청년 정책 테이블 스키마
CREATE TABLE youth_policies (
    plcy_no VARCHAR(50) PRIMARY KEY COMMENT '정책번호',
    plcy_nm VARCHAR(500) NOT NULL COMMENT '정책명',
    plcy_kywd_nm VARCHAR(200) COMMENT '정책키워드명',
    plcy_expln_cn TEXT COMMENT '정책설명내용',
    lclsf_nm VARCHAR(100) COMMENT '정책대분류명',
    mclsf_nm VARCHAR(100) COMMENT '정책중분류명',
    plcy_sprt_cn TEXT COMMENT '정책지원내용',
    
    -- 기관 정보
    sprvsn_inst_cd_nm VARCHAR(200) COMMENT '주관기관코드명',
    oper_inst_cd_nm VARCHAR(200) COMMENT '운영기관코드명',
    
    -- 지원 조건
    sprt_trgt_min_age INT COMMENT '지원대상최소연령',
    sprt_trgt_max_age INT COMMENT '지원대상최대연령',
    sprt_trgt_age_lmt_yn CHAR(1) DEFAULT 'N' COMMENT '지원대상연령제한여부',
    
    -- 신청 정보
    aply_url_addr VARCHAR(500) COMMENT '신청URL주소',
    biz_prd_bgnng_ymd VARCHAR(8) COMMENT '사업기간시작일자',
    biz_prd_end_ymd VARCHAR(8) COMMENT '사업기간종료일자',
    
    -- 요건 코드
    plcy_major_cd VARCHAR(200) COMMENT '정책전공요건코드',
    job_cd VARCHAR(200) COMMENT '정책취업요건코드',
    school_cd VARCHAR(200) COMMENT '정책학력요건코드',
    
    -- 승인 상태
    plcy_aprv_stts_cd VARCHAR(20) COMMENT '정책승인상태코드',
    
    -- 시간 정보
    frst_reg_dt DATETIME COMMENT '최초등록일시',
    
    -- 인덱스
    FULLTEXT idx_search_text (plcy_nm, plcy_expln_cn, plcy_sprt_cn)
);

-- 주요 검색 패턴:
-- 1. 텍스트 검색: MATCH() AGAINST() 또는 LIKE 사용
-- 2. 연령 조건: sprt_trgt_min_age, sprt_trgt_max_age 활용
-- 3. 카테고리: lclsf_nm, mclsf_nm 활용
-- 4. 승인 상태: plcy_aprv_stts_cd = 'APPROVED'만 조회
"""

# 카테고리별 키워드 매핑
CATEGORY_KEYWORDS = {
    PolicyCategory.SCHOLARSHIP: [
        "장학금",
        "학비",
        "등록금",
        "학자금",
        "교육비",
        "수업료",
    ],
    PolicyCategory.EMPLOYMENT: [
        "취업",
        "일자리",
        "구직",
        "채용",
        "직업",
        "고용",
        "직장",
    ],
    PolicyCategory.STARTUP: ["창업", "스타트업", "기업", "사업", "벤처", "창업자"],
    PolicyCategory.HOUSING: ["주거", "임대", "전세", "주택", "거주", "주거비", "월세"],
    PolicyCategory.LOAN: ["대출", "융자", "자금", "금융", "대여", "자금지원"],
    PolicyCategory.EDUCATION: ["교육", "연수", "훈련", "강의", "학습", "교육과정"],
    PolicyCategory.WELFARE: ["복지", "지원금", "급여", "수당", "생활비", "복지혜택"],
    PolicyCategory.MEDICAL: ["의료", "건강", "치료", "병원", "의료비", "건강검진"],
    PolicyCategory.CULTURE: ["문화", "예술", "체육", "문화활동", "예술활동"],
    PolicyCategory.GENERAL: ["지원", "혜택", "프로그램", "정책", "사업"],
}

print("✅ 데이터베이스 스키마 및 카테고리 매핑 정의 완료")

✅ 데이터베이스 스키마 및 카테고리 매핑 정의 완료


In [10]:
# cell 4: LangChain 기반 쿼리 분석기
class LangChainQueryAnalyzer:
    """LangChain과 OpenAI를 활용한 쿼리 분석기"""

    def __init__(self, model_name: str = "gpt-4o-mini"):
        self.llm = ChatOpenAI(model_name=model_name, temperature=0.1)
        self.multi_query_parser = PydanticOutputParser(
            pydantic_object=MultiQueryAnalysis
        )
        self.intent_parser = PydanticOutputParser(pydantic_object=QueryIntent)

        # 다중 쿼리 분석 프롬프트
        self.multi_query_prompt = ChatPromptTemplate.from_template(
            """
사용자의 질문을 분석하여 단일 질문인지 다중 질문인지 판단하고, 다중 질문이면 각각을 분리해주세요.

다중 질문의 패턴:
- 불릿 포인트(●, •, -, 1., 2. 등)
- 연결어(과, 와, 및, 그리고, 또한, 그리고, ,)
- 마침표로 구분된 여러 문장

사용자 질문: {user_query}

{format_instructions}
"""
        )

        # 의도 분석 프롬프트
        self.intent_prompt = ChatPromptTemplate.from_template(
            """
사용자의 질문을 분석하여 정책 카테고리와 검색 조건을 추출해주세요.

정책 카테고리:
- 장학금: 학비, 등록금, 학자금 관련
- 취업지원: 일자리, 구직, 채용, 직업 관련
- 창업지원: 창업, 스타트업, 사업 관련
- 주거지원: 주택, 임대, 전세 관련
- 대출지원: 대출, 융자, 자금 관련
- 교육지원: 교육, 연수, 훈련 관련
- 복지지원: 복지, 지원금, 수당 관련
- 의료지원: 의료, 건강, 치료 관련
- 문화지원: 문화, 예술, 체육 관련
- 일반지원: 기타 일반적인 지원

대상 그룹:
- 대학생, 청년, 고등학생, 중학생, 초등학생 등

연령 정보:
- 구체적인 나이나 연령대가 언급된 경우 추출

사용자 질문: {user_query}

{format_instructions}
"""
        )

        # 체인 생성
        self.multi_query_chain = LLMChain(
            llm=self.llm,
            prompt=self.multi_query_prompt,
            output_parser=self.multi_query_parser,
        )

        self.intent_chain = LLMChain(
            llm=self.llm, prompt=self.intent_prompt, output_parser=self.intent_parser
        )

    def analyze_multi_query(self, user_query: str) -> MultiQueryAnalysis:
        """다중 쿼리 분석"""
        try:
            with get_openai_callback() as cb:
                result = self.multi_query_chain.run(
                    user_query=user_query,
                    format_instructions=self.multi_query_parser.get_format_instructions(),
                )
                logger.info(f"다중 쿼리 분석 완료 - 토큰 사용량: {cb.total_tokens}")
                return result
        except Exception as e:
            logger.error(f"다중 쿼리 분석 실패: {e}")
            # 폴백: 간단한 규칙 기반 분석
            return self._fallback_multi_query_analysis(user_query)

    def analyze_intent(self, query: str) -> QueryIntent:
        """단일 쿼리 의도 분석"""
        try:
            with get_openai_callback() as cb:
                result = self.intent_chain.run(
                    user_query=query,
                    format_instructions=self.intent_parser.get_format_instructions(),
                )
                logger.info(f"의도 분석 완료 - 토큰 사용량: {cb.total_tokens}")
                return result
        except Exception as e:
            logger.error(f"의도 분석 실패: {e}")
            # 폴백: 간단한 규칙 기반 분석
            return self._fallback_intent_analysis(query)

    def _fallback_multi_query_analysis(self, query: str) -> MultiQueryAnalysis:
        """폴백 다중 쿼리 분석"""
        separators = ["●", "•", "과", "와", "및", "그리고", ","]
        is_multiple = any(sep in query for sep in separators)

        if is_multiple:
            # 간단한 분리 로직
            parts = (
                query.replace("●", "|")
                .replace("•", "|")
                .replace("과", "|")
                .replace("와", "|")
                .split("|")
            )
            sub_queries = [part.strip() for part in parts if part.strip()]
        else:
            sub_queries = [query]

        return MultiQueryAnalysis(
            is_multiple=is_multiple, sub_queries=sub_queries, intents=[]
        )

    def _fallback_intent_analysis(self, query: str) -> QueryIntent:
        """폴백 의도 분석"""
        # 간단한 키워드 매칭
        for category, keywords in CATEGORY_KEYWORDS.items():
            if any(keyword in query for keyword in keywords):
                return QueryIntent(
                    category=category,
                    keywords=[kw for kw in keywords if kw in query],
                    target_group=(
                        "청년"
                        if "청년" in query
                        else "대학생" if "대학생" in query else None
                    ),
                )

        return QueryIntent(category=PolicyCategory.GENERAL, keywords=["지원"])


# 쿼리 분석기 초기화
analyzer = LangChainQueryAnalyzer()
print("✅ LangChain 쿼리 분석기 초기화 완료")

✅ LangChain 쿼리 분석기 초기화 완료


In [ ]:
# cell 5: LangChain 기반 SQL 생성기
class LangChainSQLGenerator:
    """LangChain과 OpenAI를 활용한 SQL 생성기"""

    def __init__(self, model_name: str = "gpt-4o-mini"):
        self.llm = ChatOpenAI(model_name=model_name, temperature=0.1)
        self.sql_parser = PydanticOutputParser(pydantic_object=SQLQuery)

        # SQL 생성 프롬프트
        self.sql_prompt = ChatPromptTemplate.from_template(
            """
당신은 MySQL 전문가입니다. 주어진 사용자 의도와 데이터베이스 스키마를 바탕으로 최적화된 SQL 쿼리를 생성해주세요.

데이터베이스 스키마:
{schema}

사용자 의도:
- 카테고리: {category}
- 키워드: {keywords}
- 연령 범위: {age_range}
- 대상 그룹: {target_group}
- 추가 조건: {additional_conditions}

SQL 생성 규칙:
1. 반드시 승인된 정책만 조회 (plcy_aprv_stts_cd = 'APPROVED')
2. FULLTEXT 검색 우선 사용, LIKE는 보조적으로 사용
3. 연령 조건이 있으면 sprt_trgt_min_age, sprt_trgt_max_age 활용
4. 관련성 높은 순서로 정렬 (MATCH 점수 우선, 최신 등록일 순)
5. 성능을 위해 LIMIT 50 적용
6. 필요한 컬럼만 SELECT (plcy_no, plcy_nm, plcy_expln_cn, plcy_sprt_cn, aply_url_addr, sprvsn_inst_cd_nm, frst_reg_dt)

{format_instructions}
"""
        )

        # 체인 생성
        self.sql_chain = LLMChain(
            llm=self.llm, prompt=self.sql_prompt, output_parser=self.sql_parser
        )

    def generate_sql(self, intent: QueryIntent) -> SQLQuery:
        """SQL 쿼리 생성"""
        try:
            with get_openai_callback() as cb:
                result = self.sql_chain.run(
                    schema=DATABASE_SCHEMA,
                    category=intent.category.value,
                    keywords=", ".join(intent.keywords),
                    age_range=(
                        f"{intent.age_range.min_age}-{intent.age_range.max_age}"
                        if intent.age_range
                        else "제한 없음"
                    ),
                    target_group=intent.target_group or "제한 없음",
                    additional_conditions=json.dumps(
                        intent.additional_conditions, ensure_ascii=False
                    ),
                    format_instructions=self.sql_parser.get_format_instructions(),
                )
                logger.info(f"SQL 생성 완료 - 토큰 사용량: {cb.total_tokens}")
                return result
        except Exception as e:
            logger.error(f"SQL 생성 실패: {e}")
            # 폴백: 템플릿 기반 SQL 생성
            return self._fallback_sql_generation(intent)

    def _fallback_sql_generation(self, intent: QueryIntent) -> SQLQuery:
        """폴백 SQL 생성"""
        # 기본 SELECT 절
        select_columns = [
            "plcy_no",
            "plcy_nm",
            "plcy_expln_cn",
            "plcy_sprt_cn",
            "aply_url_addr",
            "sprvsn_inst_cd_nm",
            "frst_reg_dt",
        ]

        # WHERE 조건 구성
        where_conditions = ["plcy_aprv_stts_cd = 'APPROVED'"]

        # 키워드 검색 조건
        if intent.keywords:
            keyword_conditions = []
            for keyword in intent.keywords:
                keyword_conditions.extend(
                    [
                        f"plcy_nm LIKE '%{keyword}%'",
                        f"plcy_expln_cn LIKE '%{keyword}%'",
                        f"plcy_sprt_cn LIKE '%{keyword}%'",
                    ]
                )
            where_conditions.append(f"({' OR '.join(keyword_conditions)})")

        # 연령 조건
        if intent.age_range:
            age_conditions = ["sprt_trgt_age_lmt_yn = 'N'"]
            if intent.age_range.min_age and intent.age_range.max_age:
                age_conditions.append(
                    f"(sprt_trgt_min_age <= {intent.age_range.max_age} AND sprt_trgt_max_age >= {intent.age_range.min_age})"
                )
            where_conditions.append(f"({' OR '.join(age_conditions)})")

        # 대상 그룹 조건
        if intent.target_group:
            target_conditions = [
                f"plcy_expln_cn LIKE '%{intent.target_group}%'",
                f"plcy_sprt_cn LIKE '%{intent.target_group}%'",
            ]
            where_conditions.append(f"({' OR '.join(target_conditions)})")

        # 최종 쿼리 조합
        query = f"""
SELECT {', '.join(select_columns)}
FROM youth_policies
WHERE {' AND '.join(where_conditions)}
ORDER BY frst_reg_dt DESC
LIMIT 50
        """.strip()

        return SQLQuery(
            query=query,
            explanation=f"{intent.category.value} 카테고리의 정책을 검색하는 쿼리입니다.",
            estimated_results=20,
        )


# SQL 생성기 초기화
sql_generator = LangChainSQLGenerator()
print("✅ LangChain SQL 생성기 초기화 완료")

✅ LangChain SQL 생성기 초기화 완료


In [12]:
# cell 6: 메인 RAG 프로세서
class SmartRAGProcessor:
    """LangChain 기반 스마트 RAG 프로세서"""

    def __init__(self, db_config: Dict[str, str]):
        self.db_config = db_config
        self.analyzer = LangChainQueryAnalyzer()
        self.sql_generator = LangChainSQLGenerator()
        self.engine = None

    def connect_db(self):
        """데이터베이스 연결"""
        try:
            connection_string = (
                f"mysql+pymysql://{self.db_config['user']}:{self.db_config['password']}"
                f"@{self.db_config['host']}/{self.db_config['database']}"
            )
            self.engine = create_engine(connection_string)
            logger.info("✅ 데이터베이스 연결 성공")
        except Exception as e:
            logger.error(f"❌ 데이터베이스 연결 실패: {e}")
            raise

    def process_query(self, user_query: str) -> Dict[str, Any]:
        """메인 쿼리 처리 함수"""
        logger.info(f"🚀 사용자 쿼리 처리 시작: {user_query}")

        # 1. 다중 쿼리 분석
        multi_analysis = self.analyzer.analyze_multi_query(user_query)
        logger.info(f"📊 다중 쿼리 분석 결과: {multi_analysis.is_multiple}")

        results = {
            "original_query": user_query,
            "is_multiple": multi_analysis.is_multiple,
            "processing_time": datetime.now().isoformat(),
            "results": [],
        }

        # 2. 서브 쿼리 처리
        sub_queries = (
            multi_analysis.sub_queries if multi_analysis.is_multiple else [user_query]
        )

        for i, sub_query in enumerate(sub_queries, 1):
            logger.info(f"🔄 서브 쿼리 {i}/{len(sub_queries)} 처리 중: {sub_query}")

            # 2.1 의도 분석
            intent = self.analyzer.analyze_intent(sub_query)
            logger.info(f"🎯 의도 분석 결과: {intent.category.value}")

            # 2.2 SQL 생성
            sql_query = self.sql_generator.generate_sql(intent)
            logger.info(f"📝 SQL 생성 완료: {len(sql_query.query)} 문자")

            # 2.3 쿼리 실행 (DB 연결이 있는 경우)
            data = pd.DataFrame()
            if self.engine:
                try:
                    data = pd.read_sql(sql_query.query, self.engine)
                    logger.info(f"✅ 쿼리 실행 성공: {len(data)}개 결과")
                except Exception as e:
                    logger.error(f"❌ 쿼리 실행 실패: {e}")

            # 결과 저장
            results["results"].append(
                {
                    "sub_query": sub_query,
                    "intent": intent.dict(),
                    "sql_query": sql_query.query,
                    "sql_explanation": sql_query.explanation,
                    "data": data.to_dict("records") if not data.empty else [],
                    "count": len(data),
                }
            )

        return results

    def explain_query(self, user_query: str) -> Dict[str, Any]:
        """쿼리 처리 과정 설명 (디버깅용)"""
        logger.info(f"🔍 쿼리 분석 모드: {user_query}")

        # 다중 쿼리 분석
        multi_analysis = self.analyzer.analyze_multi_query(user_query)

        explanation = {
            "original_query": user_query,
            "is_multiple": multi_analysis.is_multiple,
            "sub_queries": multi_analysis.sub_queries,
            "analysis_details": [],
        }

        for sub_query in multi_analysis.sub_queries:
            # 의도 분석
            intent = self.analyzer.analyze_intent(sub_query)

            # SQL 생성
            sql_query = self.sql_generator.generate_sql(intent)

            explanation["analysis_details"].append(
                {
                    "sub_query": sub_query,
                    "category": intent.category.value,
                    "keywords": intent.keywords,
                    "age_range": intent.age_range.dict() if intent.age_range else None,
                    "target_group": intent.target_group,
                    "generated_sql": sql_query.query,
                    "sql_explanation": sql_query.explanation,
                }
            )

        return explanation


# 테스트용 DB 설정
TEST_DB_CONFIG = {
    "host": os.environ["DB_HOST"],
    "user": os.environ["DB_USER"],
    "password": os.environ["DB_PASSWORD"],
    "database": os.environ["DB_NAME"],
    "charset": "utf8mb4",
}

print(f"DB_CONFIG: {TEST_DB_CONFIG}")

# 프로세서 초기화
processor = SmartRAGProcessor(TEST_DB_CONFIG)
print("✅ 스마트 RAG 프로세서 초기화 완료")

DB_CONFIG: {'host': 'localhost', 'user': 'steve', 'password': 'doolman', 'database': 'youth_policy_db', 'charset': 'utf8mb4'}
✅ 스마트 RAG 프로세서 초기화 완료


In [9]:
# cell 7: 테스트 케이스 정의 및 실행
# 10개의 예상 질문 정의
TEST_QUERIES = [
    "대학생 장학금과 청년 취업 지원 프로그램 정보 알려줘",
    "● 대학생 장학금 정보 알려줘. ● 청년 취업 지원 프로그램 정보 알려줘.",
    "20대 청년 창업 지원 정책 찾아줘",
    "고등학생 교육 지원 프로그램 있어?",
    "주거 지원과 의료 지원 정책 알려줘",
    "25세 이하 대출 지원 프로그램 찾아줘",
    "문화 활동 지원 정책 및 복지 지원 정책 알려줘",
    "대학생 해외 연수 지원 프로그램 있나요?",
    "● 청년 일자리 지원 ● 창업 자금 지원 ● 주거 지원",
    "30세 미만 청년 종합 지원 정책 알려줘",
]


def run_test_cases():
    """테스트 케이스 실행"""
    print("🧪 테스트 케이스 실행 시작")
    print("=" * 100)

    for i, query in enumerate(TEST_QUERIES, 1):
        print(f"\n📋 테스트 케이스 {i}/10")
        print(f"   질문: {query}")
        print("-" * 80)

        try:
            # 쿼리 분석 (실제 DB 연결 없이)
            explanation = processor.explain_query(query)

            print(f"   🔍 분석 결과:")
            print(
                f"      - 다중 쿼리: {'예' if explanation['is_multiple'] else '아니오'}"
            )
            print(f"      - 서브 쿼리 수: {len(explanation['sub_queries'])}")

            for j, detail in enumerate(explanation["analysis_details"], 1):
                print(f"      - 서브 쿼리 {j}: {detail['sub_query']}")
                print(f"        · 카테고리: {detail['category']}")
                print(f"        · 키워드: {detail['keywords']}")
                print(f"        · 대상 그룹: {detail['target_group']}")
                print(f"        · SQL 설명: {detail['sql_explanation']}")
                print(f"        · SQL 미리보기: {detail['generated_sql'][:100]}...")

        except Exception as e:
            print(f"   ❌ 테스트 실패: {e}")

        print("-" * 80)

    print("\n🎉 모든 테스트 케이스 완료!")


# 테스트 실행
run_test_cases()

2025-07-18 17:08:40,320 - INFO - 🔍 쿼리 분석 모드: 대학생 장학금과 청년 취업 지원 프로그램 정보 알려줘


🧪 테스트 케이스 실행 시작

📋 테스트 케이스 1/10
   질문: 대학생 장학금과 청년 취업 지원 프로그램 정보 알려줘
--------------------------------------------------------------------------------


C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_13712\1947969847.py:72: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = self.multi_query_chain.run(
2025-07-18 17:08:42,751 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-07-18 17:08:42,773 - INFO - 다중 쿼리 분석 완료 - 토큰 사용량: 1146
2025-07-18 17:08:42,989 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
2025-07-18 17:08:42,991 - INFO - Retrying request to /chat/completions in 8.640000 seconds
2025-07-18 17:08:52,872 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-07-18 17:08:52,875 - INFO - 의도 분석 완료 - 토큰 사용량: 1073
2025-07-18 17:08:53,236 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
2025-07-18 17:08:53,238 - INFO - Retrying reques

   🔍 분석 결과:
      - 다중 쿼리: 예
      - 서브 쿼리 수: 2
      - 서브 쿼리 1: 대학생 장학금 정보 알려줘
        · 카테고리: 장학금
        · 키워드: ['대학생', '장학금', '정보']
        · 대상 그룹: 대학생
        · SQL 설명: 장학금 카테고리의 정책을 검색하는 쿼리입니다.
        · SQL 미리보기: SELECT plcy_no, plcy_nm, plcy_expln_cn, plcy_sprt_cn, aply_url_addr, sprvsn_inst_cd_nm, frst_reg_dt
...
      - 서브 쿼리 2: 청년 취업 지원 프로그램 정보 알려줘
        · 카테고리: 취업지원
        · 키워드: ['프로그램', '정보']
        · 대상 그룹: 청년
        · SQL 설명: 취업지원 카테고리의 정책을 검색하는 쿼리입니다.
        · SQL 미리보기: SELECT plcy_no, plcy_nm, plcy_expln_cn, plcy_sprt_cn, aply_url_addr, sprvsn_inst_cd_nm, frst_reg_dt
...
--------------------------------------------------------------------------------

📋 테스트 케이스 2/10
   질문: ● 대학생 장학금 정보 알려줘. ● 청년 취업 지원 프로그램 정보 알려줘.
--------------------------------------------------------------------------------


2025-07-18 17:09:41,850 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-07-18 17:09:41,864 - INFO - 다중 쿼리 분석 완료 - 토큰 사용량: 1145
2025-07-18 17:09:43,150 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-07-18 17:09:43,156 - INFO - 의도 분석 완료 - 토큰 사용량: 1073
2025-07-18 17:09:46,078 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-07-18 17:09:46,089 - INFO - SQL 생성 완료 - 토큰 사용량: 1413
2025-07-18 17:09:47,018 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-07-18 17:09:47,025 - INFO - 의도 분석 완료 - 토큰 사용량: 1072
2025-07-18 17:09:47,226 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
2025-07-18 17:09:47,227 - INFO - Retrying request to /chat/completions in 8.640000 seconds
2025-07-18 17:09:56,276 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "H

...

2025-07-18 17:14:43,926 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-07-18 17:14:43,929 - INFO - 다중 쿼리 분석 완료 - 토큰 사용량: 1162
2025-07-18 17:14:44,136 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
2025-07-18 17:14:44,137 - INFO - Retrying request to /chat/completions in 8.640000 seconds
2025-07-18 17:14:53,953 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-07-18 17:14:53,955 - INFO - 의도 분석 완료 - 토큰 사용량: 1060
2025-07-18 17:14:59,322 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-07-18 17:14:59,330 - INFO - SQL 생성 완료 - 토큰 사용량: 1487
2025-07-18 17:14:59,691 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
2025-07-18 17:14:59,692 - INFO - Retrying request to /chat/completions in 8.640000 seconds
2025-07-18 17:15:08,555 - INFO - HTTP Request: P

   🔍 분석 결과:
      - 다중 쿼리: 예
      - 서브 쿼리 수: 3
      - 서브 쿼리 1: 청년 일자리 지원
        · 카테고리: 취업지원
        · 키워드: ['청년', '일자리']
        · 대상 그룹: 청년
        · SQL 설명: 카테고리가 '취업지원'이고, 키워드에 '청년'과 '일자리'가 포함된 승인된 정책을 관련성 높은 순서로 최신 등록일 순으로 조회합니다. 연령 제한이 없는 정책을 대상으로 하며, 최대 50개의 결과를 반환합니다.
        · SQL 미리보기: SELECT plcy_no, plcy_nm, plcy_expln_cn, plcy_sprt_cn, aply_url_addr, sprvsn_inst_cd_nm, frst_reg_dt ...
      - 서브 쿼리 2: 창업 자금 지원
        · 카테고리: 창업지원
        · 키워드: ['자금', '지원']
        · 대상 그룹: None
        · SQL 설명: 카테고리가 '창업지원'이고 키워드에 '자금' 또는 '지원'이 포함된 승인된 정책을 관련성 높은 순서로 최대 50개까지 조회합니다.
        · SQL 미리보기: SELECT plcy_no, plcy_nm, plcy_expln_cn, plcy_sprt_cn, aply_url_addr, sprvsn_inst_cd_nm, frst_reg_dt ...
      - 서브 쿼리 3: 주거 지원
        · 카테고리: 주거지원
        · 키워드: []
        · 대상 그룹: None
        · SQL 설명: 주거지원 카테고리에 해당하는 승인된 정책을 관련성 높은 순서로 최신 등록일 순으로 조회합니다. 연령 제한이 없는 정책만을 대상으로 합니다.
        · SQL 미리보기: SELECT plcy_no, plcy_nm, plcy_expln_cn, plcy_sprt_cn, aply_url_addr, sprvsn_inst_cd_

2025-07-18 17:16:01,188 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-07-18 17:16:01,191 - INFO - 다중 쿼리 분석 완료 - 토큰 사용량: 1111
2025-07-18 17:16:02,309 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-07-18 17:16:02,323 - INFO - 의도 분석 완료 - 토큰 사용량: 1084
2025-07-18 17:16:04,353 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-07-18 17:16:04,356 - INFO - SQL 생성 완료 - 토큰 사용량: 1457


   🔍 분석 결과:
      - 다중 쿼리: 예
      - 서브 쿼리 수: 1
      - 서브 쿼리 1: 30세 미만 청년 종합 지원 정책 알려줘
        · 카테고리: 일반지원
        · 키워드: ['청년', '종합 지원']
        · 대상 그룹: 청년
        · SQL 설명: 카테고리가 '일반지원'이고, 키워드에 '청년 종합 지원'이 포함되며, 연령이 30세 이하인 승인된 정책을 관련성과 최신 등록일 순으로 조회합니다.
        · SQL 미리보기: SELECT plcy_no, plcy_nm, plcy_expln_cn, plcy_sprt_cn, aply_url_addr, sprvsn_inst_cd_nm, frst_reg_dt ...
--------------------------------------------------------------------------------

🎉 모든 테스트 케이스 완료!


C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_13712\42994883.py:104: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  "age_range": intent.age_range.dict() if intent.age_range else None,


In [13]:
# cell 8: 실제 데이터베이스 연결 테스트 (선택적)
def test_with_real_database():
    """실제 데이터베이스 연결 테스트"""
    print("🔗 실제 데이터베이스 연결 테스트")
    print("⚠️  실제 DB 연결 정보를 TEST_DB_CONFIG에 설정해야 합니다.")

    try:
        # 데이터베이스 연결
        processor.connect_db()

        # 테스트 쿼리 실행
        test_query = "대학생 장학금 정보 알려줘"
        result = processor.process_query(test_query)

        print(f"✅ 테스트 성공!")
        print(f"   - 원본 쿼리: {result['original_query']}")
        print(f"   - 다중 쿼리 여부: {result['is_multiple']}")
        print(f"   - 결과 수: {len(result['results'])}")

        for i, res in enumerate(result["results"], 1):
            print(f"   - 서브 결과 {i}:")
            print(f"     · 쿼리: {res['sub_query']}")
            print(f"     · 카테고리: {res['intent']['category']}")
            print(f"     · 데이터 건수: {res['count']}")

            if res["count"] > 0:
                print(f"     · 첫 번째 결과: {res['data'][0]['plcy_nm']}")

    except Exception as e:
        print(f"❌ 데이터베이스 연결 실패: {e}")
        print("💡 DB_CONFIG를 실제 환경에 맞게 설정하세요")


# 실제 DB 연결 테스트 (주석 해제하여 사용)
test_with_real_database()
# print("💡 실제 데이터베이스 테스트를 위해 test_with_real_database() 함수를 실행하세요")

2025-07-18 17:26:11,797 - INFO - ✅ 데이터베이스 연결 성공
2025-07-18 17:26:11,799 - INFO - 🚀 사용자 쿼리 처리 시작: 대학생 장학금 정보 알려줘


🔗 실제 데이터베이스 연결 테스트
⚠️  실제 DB 연결 정보를 TEST_DB_CONFIG에 설정해야 합니다.


2025-07-18 17:26:12,937 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-07-18 17:26:12,954 - INFO - 다중 쿼리 분석 완료 - 토큰 사용량: 848
2025-07-18 17:26:12,955 - INFO - 📊 다중 쿼리 분석 결과: False
2025-07-18 17:26:12,956 - INFO - 🔄 서브 쿼리 1/1 처리 중: 대학생 장학금 정보 알려줘
2025-07-18 17:26:14,585 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-07-18 17:26:14,590 - INFO - 의도 분석 완료 - 토큰 사용량: 855
2025-07-18 17:26:14,592 - INFO - 🎯 의도 분석 결과: 장학금
2025-07-18 17:26:16,158 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-07-18 17:26:16,163 - INFO - SQL 생성 완료 - 토큰 사용량: 1414
2025-07-18 17:26:16,165 - INFO - 📝 SQL 생성 완료: 272 문자
2025-07-18 17:26:16,233 - INFO - ✅ 쿼리 실행 성공: 0개 결과


✅ 테스트 성공!
   - 원본 쿼리: 대학생 장학금 정보 알려줘
   - 다중 쿼리 여부: False
   - 결과 수: 1
   - 서브 결과 1:
     · 쿼리: 대학생 장학금 정보 알려줘
     · 카테고리: PolicyCategory.SCHOLARSHIP
     · 데이터 건수: 0


C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_13712\1220425489.py:68: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  "intent": intent.dict(),


In [14]:
# cell 9: 성능 분석 및 최적화
import time
from collections import defaultdict


class PerformanceAnalyzer:
    """성능 분석기"""

    def __init__(self):
        self.metrics = defaultdict(list)

    def benchmark_queries(self, queries: List[str], iterations: int = 3):
        """쿼리 성능 벤치마크"""
        print(f"⏱️  성능 벤치마크 시작 (반복: {iterations}회)")

        for query in queries:
            print(f"🔍 테스트 중: {query[:50]}...")

            times = []
            for i in range(iterations):
                start_time = time.time()

                try:
                    # 쿼리 분석만 수행 (DB 연결 없이)
                    explanation = processor.explain_query(query)
                    elapsed = time.time() - start_time
                    times.append(elapsed)

                except Exception as e:
                    print(f"   ❌ 오류: {e}")
                    continue

            if times:
                avg_time = sum(times) / len(times)
                self.metrics[query] = {
                    "avg_time": avg_time,
                    "min_time": min(times),
                    "max_time": max(times),
                    "iterations": len(times),
                }
                print(f"   ✅ 평균 처리 시간: {avg_time:.3f}초")

        self.print_summary()

    def print_summary(self):
        """성능 요약 출력"""
        print("\n📊 성능 분석 요약")
        print("=" * 80)

        if not self.metrics:
            print("   측정된 데이터가 없습니다.")
            return

        all_times = [m["avg_time"] for m in self.metrics.values()]

        print(f"   📈 전체 평균 처리 시간: {sum(all_times)/len(all_times):.3f}초")
        print(f"   ⚡ 최고 속도: {min(all_times):.3f}초")
        print(f"   🐌 최저 속도: {max(all_times):.3f}초")

        print("\n   📋 쿼리별 상세 결과:")
        for query, metrics in sorted(
            self.metrics.items(), key=lambda x: x[1]["avg_time"]
        ):
            print(f"      {query[:60]}... : {metrics['avg_time']:.3f}초")


# 성능 분석 실행
analyzer = PerformanceAnalyzer()
sample_queries = TEST_QUERIES[:5]  # 처음 5개 쿼리만 테스트
analyzer.benchmark_queries(sample_queries)

2025-07-18 17:27:56,806 - INFO - 🔍 쿼리 분석 모드: 대학생 장학금과 청년 취업 지원 프로그램 정보 알려줘


⏱️  성능 벤치마크 시작 (반복: 3회)
🔍 테스트 중: 대학생 장학금과 청년 취업 지원 프로그램 정보 알려줘...


2025-07-18 17:27:59,884 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-07-18 17:27:59,903 - INFO - 다중 쿼리 분석 완료 - 토큰 사용량: 991
2025-07-18 17:28:01,842 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-07-18 17:28:01,850 - INFO - 의도 분석 완료 - 토큰 사용량: 855
2025-07-18 17:28:03,549 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-07-18 17:28:03,557 - INFO - SQL 생성 완료 - 토큰 사용량: 1424
2025-07-18 17:28:06,938 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-07-18 17:28:06,944 - INFO - 의도 분석 완료 - 토큰 사용량: 862
2025-07-18 17:28:08,733 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-07-18 17:28:08,738 - INFO - SQL 생성 완료 - 토큰 사용량: 1489
2025-07-18 17:28:08,739 - INFO - 🔍 쿼리 분석 모드: 대학생 장학금과 청년 취업 지원 프로그램 정보 알려줘
2025-07-18 17:28:11,835 - INFO - HTTP Request: POST https://api.openai.co

   ✅ 평균 처리 시간: 13.327초
🔍 테스트 중: ● 대학생 장학금 정보 알려줘. ● 청년 취업 지원 프로그램 정보 알려줘....


2025-07-18 17:28:41,207 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-07-18 17:28:41,215 - INFO - 다중 쿼리 분석 완료 - 토큰 사용량: 1007
2025-07-18 17:28:42,560 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-07-18 17:28:42,564 - INFO - 의도 분석 완료 - 토큰 사용량: 855
2025-07-18 17:28:44,508 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-07-18 17:28:44,519 - INFO - SQL 생성 완료 - 토큰 사용량: 1423
2025-07-18 17:28:46,241 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-07-18 17:28:46,246 - INFO - 의도 분석 완료 - 토큰 사용량: 859
2025-07-18 17:28:49,239 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-07-18 17:28:49,249 - INFO - SQL 생성 완료 - 토큰 사용량: 1464
2025-07-18 17:28:49,253 - INFO - 🔍 쿼리 분석 모드: ● 대학생 장학금 정보 알려줘. ● 청년 취업 지원 프로그램 정보 알려줘.
2025-07-18 17:28:52,577 - INFO - HTTP Request: POST https://

...

   ✅ 평균 처리 시간: 2.802초
🔍 테스트 중: 고등학생 교육 지원 프로그램 있어?...


2025-07-18 17:29:24,597 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-07-18 17:29:24,610 - INFO - 다중 쿼리 분석 완료 - 토큰 사용량: 847
2025-07-18 17:29:24,611 - INFO - 🔍 쿼리 분석 모드: 고등학생 교육 지원 프로그램 있어?
2025-07-18 17:29:25,445 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-07-18 17:29:25,452 - INFO - 다중 쿼리 분석 완료 - 토큰 사용량: 847
2025-07-18 17:29:25,453 - INFO - 🔍 쿼리 분석 모드: 고등학생 교육 지원 프로그램 있어?
2025-07-18 17:29:26,346 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-07-18 17:29:26,353 - INFO - 다중 쿼리 분석 완료 - 토큰 사용량: 847
2025-07-18 17:29:26,354 - INFO - 🔍 쿼리 분석 모드: 주거 지원과 의료 지원 정책 알려줘


   ✅ 평균 처리 시간: 1.028초
🔍 테스트 중: 주거 지원과 의료 지원 정책 알려줘...


2025-07-18 17:29:29,477 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-07-18 17:29:29,485 - INFO - 다중 쿼리 분석 완료 - 토큰 사용량: 959
2025-07-18 17:29:30,564 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-07-18 17:29:30,602 - INFO - 의도 분석 완료 - 토큰 사용량: 840
2025-07-18 17:29:37,335 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-07-18 17:29:37,347 - INFO - SQL 생성 완료 - 토큰 사용량: 1422
2025-07-18 17:29:38,994 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-07-18 17:29:39,002 - INFO - 의도 분석 완료 - 토큰 사용량: 849
2025-07-18 17:29:40,787 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-07-18 17:29:40,799 - INFO - SQL 생성 완료 - 토큰 사용량: 1451
2025-07-18 17:29:40,803 - INFO - 🔍 쿼리 분석 모드: 주거 지원과 의료 지원 정책 알려줘
2025-07-18 17:29:43,931 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/

   ✅ 평균 처리 시간: 16.359초

📊 성능 분석 요약
   📈 전체 평균 처리 시간: 9.242초
   ⚡ 최고 속도: 1.028초
   🐌 최저 속도: 16.359초

   📋 쿼리별 상세 결과:
      고등학생 교육 지원 프로그램 있어?... : 1.028초
      20대 청년 창업 지원 정책 찾아줘... : 2.802초
      ● 대학생 장학금 정보 알려줘. ● 청년 취업 지원 프로그램 정보 알려줘.... : 12.692초
      대학생 장학금과 청년 취업 지원 프로그램 정보 알려줘... : 13.327초
      주거 지원과 의료 지원 정책 알려줘... : 16.359초


In [15]:
# cell 10: 고급 기능 및 유틸리티
class AdvancedFeatures:
    """고급 기능 모음"""

    def __init__(self, processor: SmartRAGProcessor):
        self.processor = processor

    def generate_sql_documentation(self, queries: List[str], output_file: str = "sql_documentation.md"):
        """SQL 쿼리 문서 생성"""
        print(f"📄 SQL 문서 생성 중: {output_file}")

        with open(output_file, 'w', encoding='utf-8') as f:
            f.write("# 온통청년 RAG 시스템 SQL 쿼리 문서\n\n")
            f.write(f"생성일시: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n\n")

            for i, query in enumerate(queries, 1):
                f.write(f"## 쿼리 {i}: {query}\n\n")

                try:
                    explanation = self.processor.explain_query(query)

                    f.write(f"**다중 쿼리 여부:** {'예' if explanation['is_multiple'] else '아니오'}\n\n")

                    for j, detail in enumerate(explanation['analysis_details'], 1):
                        f.write(f"### 서브 쿼리 {j}\n")
                        f.write(f"- **쿼리:** {detail['sub_query']}\n")
                        f.write(f"- **카테고리:** {detail['category']}\n")
                        f.write(f"- **키워드:** {', '.join(detail['keywords'])}\n")
                        f.write(f"- **대상 그룹:** {detail['target_group'] or '없음'}\n")
                        f.write(f"- **설명:** {detail['sql_explanation']}\n\n")
                        f.write("```\n\n")
                        f.write(detail['generated_sql'])
                        f.write("\n```\n\n")

                except Exception as e:
                    f.write(f"❌ 처리 실패: {e}\n\n")

                f.write("---\n\n")

        print(f"✅ 문서 생성 완료: {output_file}")

    def export_queries_json(self, queries: List[str], output_file: str = "queries.json"):
        """쿼리 분석 결과를 JSON으로 내보내기"""
        print(f"📦 JSON 내보내기 중: {output_file}")

        results = []
        for query in queries:
            try:
                explanation = self.processor.explain_query(query)
                results.append(explanation)
            except Exception as e:
                results.append({
                    'original_query': query,
                    'error': str(e)
                })

        with open(output_file, 'w', encoding='utf-8') as f:
            json.dump(results, f, ensure_ascii=False, indent=2)

        print(f"✅ JSON 내보내기 완료: {output_file}")

    def analyze_query_patterns(self, queries: List[str]):
        """쿼리 패턴 분석"""
        print("🔍 쿼리 패턴 분석")

        patterns = {
            'categories': defaultdict(int),
            'keywords': defaultdict(int),
            'target_groups': defaultdict(int),
            'multi_query_count': 0
        }

        for query in queries:
            try:
                explanation = self.processor.explain_query(query)

                if explanation['is_multiple']:
                    patterns['multi_query_count'] += 1

                for detail in explanation['analysis_details']:
                    patterns['categories'][detail['category']] += 1

                    for keyword in detail['keywords']:
                        patterns['keywords'][keyword] += 1

                    if detail['target_group']:
                        patterns['target_groups'][detail['target_group']] += 1

            except Exception as e:
                print(f"   ❌ 분석 실패: {query} - {e}")

        print(f"\n📊 패턴 분석 결과:")
        print(f"   - 다중 쿼리: {patterns['multi_query_count']}/{len(queries)}개")
        print(f"   - 주요 카테고리: {dict(patterns['categories'])}")
        print(f"   - 빈출 키워드: {dict(list(patterns['keywords'].most_common(5)))}")
        print(f"   - 대상 그룹: {dict(patterns['target_groups'])}")

# 고급 기능 테스트
advanced = AdvancedFeatures(processor)

# 패턴 분석 실행
advanced.analyze_query_patterns(TEST_QUERIES)

# 문서 생성 (선택적)
# advanced.generate_sql_documentation(TEST_QUERIES[:3])
# advanced.export_queries_json(TEST_QUERIES[:3])

print("\n✅ 모든 기능 테스트 완료!")
print("📚 추가 기능:")
print("   - generate_sql_documentation(): SQL 문서 생성")
print("   - export_queries_json(): JSON 내보내기")
print("   - analyze_query_patterns(): 쿼리 패턴 분석")

2025-07-18 17:33:07,359 - INFO - 🔍 쿼리 분석 모드: 대학생 장학금과 청년 취업 지원 프로그램 정보 알려줘


🔍 쿼리 패턴 분석


2025-07-18 17:33:10,569 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-07-18 17:33:10,583 - INFO - 다중 쿼리 분석 완료 - 토큰 사용량: 991
2025-07-18 17:33:14,014 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-07-18 17:33:14,020 - INFO - 의도 분석 완료 - 토큰 사용량: 855
2025-07-18 17:33:16,579 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-07-18 17:33:16,584 - INFO - SQL 생성 완료 - 토큰 사용량: 1428
2025-07-18 17:33:20,564 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-07-18 17:33:20,572 - INFO - 의도 분석 완료 - 토큰 사용량: 859
2025-07-18 17:33:22,824 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-07-18 17:33:22,827 - INFO - SQL 생성 완료 - 토큰 사용량: 1496
2025-07-18 17:33:22,828 - INFO - 🔍 쿼리 분석 모드: ● 대학생 장학금 정보 알려줘. ● 청년 취업 지원 프로그램 정보 알려줘.
2025-07-18 17:33:25,881 - INFO - HTTP Request: POST https://a


📊 패턴 분석 결과:
   - 다중 쿼리: 5/10개
   - 주요 카테고리: {'장학금': 2, '취업지원': 3, '주거지원': 2, '의료지원': 1, '문화지원': 1, '복지지원': 1, '창업지원': 1}


AttributeError: 'collections.defaultdict' object has no attribute 'most_common'